### Train model based on the different samples



Use same strategy as in the annomatic paper.

Test on BABE


In [1]:
!pip install vendi_score

import logging
import warnings
import os
import pandas as pd
import numpy as np
from vendi_score import vendi
import torch
import transformers
from datasets import Dataset,load_dataset
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    f1_score,
    matthews_corrcoef,
    precision_score,
    recall_score,
    roc_auc_score,
)

transformers.logging.set_verbosity(transformers.logging.ERROR)
logging.disable(logging.ERROR)
warnings.filterwarnings("ignore", category=FutureWarning)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for vendi_score: filename=vendi_score-0.0.3-py3-none-any.whl size=13316 sha256=a6f890a8646f8605b4cf9178167da288c0b711afcadefc051b29911041578222
  Stored in directory: /root/.cache/pip/wheels/61/3e/72/6111f6ae4d7517f7d29a0bb1699bf6ea43ed48ff96bb10d723
Successfully built vendi_score


In [2]:
# Load subsamples
sampled_dfs = {}

samples_folder = "/kaggle/input/dirichlet-source-party/subsamples"

for filename in os.listdir(samples_folder):
    if filename.endswith(".csv"):
        sample_name = filename.split(".")[0]
        sampled_dfs[sample_name] = pd.read_csv(os.path.join(samples_folder, filename))



# Display the keys of the sampled_dfs dictionary to verify the loaded subsamples
print(sampled_dfs.keys())

dict_keys(['alpha_10_idx_2', 'alpha_10_idx_3', 'alpha_1_idx_1', 'alpha_100_idx_3', 'alpha_10_idx_1', 'alpha_1000_idx_1', 'alpha_0_01_idx_3', 'alpha_1_idx_2', 'alpha_100_idx_2', 'alpha_inf_idx_2', 'alpha_100_idx_1', 'alpha_inf_idx_3', 'alpha_1000_idx_2', 'alpha_0_1_idx_1', 'alpha_0_01_idx_2', 'alpha_0_01_idx_1', 'alpha_0_1_idx_3', 'alpha_inf_idx_1', 'alpha_1_idx_3', 'alpha_1000_idx_3', 'alpha_0_1_idx_2'])


In [3]:
# Define Vendi Score function for diversity measure

def vs_source(df):
    features= df[['reliability', 'bias']].to_numpy()
    distances= np.sqrt(np.sum((features[:, np.newaxis] - features[np.newaxis, :]) ** 2, axis=-1))
    similarity_matrix = 1 / (1 + distances)

    vs = vendi.score_K(similarity_matrix)
    return vs



for name, df in sampled_dfs.items():
    # VS score
    print(vs_source(df))

42.36204969411056
41.89943211470747
29.0637241180812
42.23270525836314
40.77482204321009
42.46382987405022
8.810501531372692
26.742814429553615
42.372671618394776
43.77923979483535
41.0264544016782
42.311102025241894
40.88155173576922
8.751802312073396
8.42884902837399
12.809857865035188
18.217655212962526
42.12518033568915
29.666980490335277
41.61326489845395
21.20190529122967


Fine-tune Roberta model on the different subsamples

In [4]:
# Definitions

base_model = "roberta-base"
device = (
    torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
)

# Prepare Model
model = AutoModelForSequenceClassification.from_pretrained(
    base_model,
    num_labels=2,
)
tokenizer = AutoTokenizer.from_pretrained(base_model)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # dynammically pads sequences to the length of the longest sequence in each batch


# the pretrained head of roberta is discarded and replaced with a randomly initialized classification head

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
# Define metrics
def compute_metrics_hf(preds):
    """computes F1 score and accuracy over dataset
    Args:
        model (any type): model for evaluation
        testing_dataloader (huggingface dataset): self explained
    Returns:
        dict
    """
    y_pred=preds.predictions
    y_true=preds.label_ids
    y_pred=y_pred.argmax(axis=1)
    mcc=matthews_corrcoef(y_true,y_pred)
    f1=f1_score(y_true,y_pred)
    precision=precision_score(y_true,y_pred)
    recall=recall_score(y_true,y_pred)
    roc_auc=roc_auc_score(y_true,y_pred)
    return{
        "mcc":mcc,
        "f1":f1,
        "precision":precision,
        "recall":recall,
        "roc_auc":roc_auc,
    }

def compute_metrics(testing_dataloader,model):
    """computes F1 score and accuracy over dataset
    Args:
        model (any type): model for evaluation
        testing_dataloader (huggingface dataset): self explained
    Returns:
        dict
    """
    y_true=[]
    y_pred=[]
    model.eval()
    for batch in testing_dataloader:
        batch={k:v.to(model.device)for k,v in batch.items()}
        with torch.no_grad():
            outputs=model(**batch)
        logits=outputs.logits
        predictions=torch.argmax(logits,dim=-1)
        y_true.extend(batch["labels"].tolist())
        y_pred.extend(predictions.tolist())
    mcc=matthews_corrcoef(y_true,y_pred)
    f1=f1_score(y_true,y_pred)
    precision=precision_score(y_true,y_pred)
    recall=recall_score(y_true,y_pred)
    roc_auc=roc_auc_score(y_true,y_pred)
    return{
        "mcc":mcc,
        "f1":f1,
        "precision":precision,
        "recall":recall,
        "roc_auc":roc_auc,
    }


In [7]:
# metrics and vs scores
results_list=[]

def tokenize_function(examples):
    return tokenizer(examples['text'], padding=True, truncation=True, max_length=128, return_tensors="pt")

# Import BABE for test sets
babe= load_dataset("mediabiasgroup/BABE")["train"].to_pandas()

dev_df, test_df = train_test_split(babe,test_size=0.67,random_state=42,stratify=babe['label'])
dev_df = Dataset.from_pandas(dev_df).map(tokenize_function, batched=True)
test_df = Dataset.from_pandas(test_df).map(tokenize_function, batched=True)

dev_df.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_df.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])



# loop through each sampled df and evaluate the model
for name,df in sampled_dfs.items():
    # VS score
    df_vs= vs_source(df)
    
    # Convert to hf datasets
    train_df=Dataset.from_pandas(df).map(tokenize_function,batched=True)
    train_df.set_format(type='torch',columns=['input_ids','attention_mask','label'])

    
    training_args=TrainingArguments(
        output_dir="./checkpoints",
        per_device_eval_batch_size=32,
        per_device_train_batch_size=32,
        num_train_epochs=3,
        save_total_limit=3,
        evaluation_strategy="steps",
        logging_steps=50,
        eval_steps=50,
        save_steps=50,
        disable_tqdm=False,
        weight_decay=0.05,
        learning_rate=2e-5,
        # run_name="annomatic_test_eval",
        metric_for_best_model="eval_loss",
        save_strategy="steps",
        load_best_model_at_end=True,
        remove_unused_columns=False,
    )

    # Trainer initialization
    trainer=Trainer(
        model,
        training_args,
        train_dataset=train_df,
        eval_dataset=dev_df,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_hf,
        data_collator=data_collator,
        callbacks=[
            EarlyStoppingCallback(
                early_stopping_patience=5,
                early_stopping_threshold=0.0,
            ),
        ],
    )

    # Train the model
    trainer.train()
    
    # Evaluate on the test set
    eval_dataloader=DataLoader(
        test_df,
        batch_size=32,
        collate_fn=data_collator,
    )
    
    metrics=compute_metrics(eval_dataloader,model)
    metrics['vs']=df_vs  # Add the vs score to the metrics
    metrics['subsample']=name  # Add the df name   
    results_list.append(metrics)


Map:   0%|          | 0/1029 [00:00<?, ? examples/s]

Map:   0%|          | 0/2092 [00:00<?, ? examples/s]

Map:   0%|          | 0/2998 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


...

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Map:   0%|          | 0/2999 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Mcc,F1,Precision,Recall,Roc Auc
50,0.320600,0.402045,0.644182,0.846416,0.829431,0.864111,0.819968
100,0.168500,0.545562,0.610266,0.833755,0.808511,0.860627,0.801742


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [ ]:
results_df = pd.DataFrame(results_list)
results_df = results_df[['subsample'] + [col for col in results_df.columns if col != 'subsample']]

results['alpha'] = results['subsample'].str.extract(r'alpha_([\d_]+)').replace('_', '.', regex=True)
results_df.to_csv("results_df.csv", index=False)


# df['alpha'] = df['name'].str.extract(r'alpha_([\d_]+)').replace('_', '.', regex=True)


# styled_df = test.style \
#     .highlight_max(color='lightgreen', axis=0, subset=results_df.columns.difference(['subsample'])) \
#     .highlight_min(color='lightcoral', axis=0, subset=results_df.columns.difference(['subsample']))



# # Display the styled DataFrame

# # styled_df.sort_values(by=["vs"])
# styled_df


In [21]:
results_df = pd.DataFrame(results_list)
results_df = results_df[['subsample'] + [col for col in results_df.columns if col != 'subsample']]

results_df['alpha'] = results_df['subsample'].str.extract(r'alpha_(.*?)_idx').replace('_', '.', regex=True)
results_df['alpha'] = results_df['alpha'].replace(np.inf, float('inf'))

results_df = results_df.sort_values(by='alpha', ascending=True)

results_df.to_csv("results_df.csv", index=False)

results_df

,subsample,mcc,f1,precision,recall,roc_auc,vs,alpha
15,alpha_0_01_idx_1,0.664689,0.855225,0.837305,0.873928,0.830053,12.809858,0.01
14,alpha_0_01_idx_2,0.669845,0.851691,0.861404,0.842196,0.835785,8.428849,0.01
6,alpha_0_01_idx_3,0.623381,0.843750,0.791729,0.903087,0.801976,8.810502,0.01
20,alpha_0_1_idx_2,0.657920,0.851852,0.836364,0.867925,0.827051,21.201905,0.1
16,alpha_0_1_idx_3,0.662145,0.852292,0.843697,0.861063,0.830100,18.217655,0.1
13,alpha_0_1_idx_1,0.654072,0.849916,0.835821,0.864494,0.825336,8.751802,0.1
18,alpha_1_idx_3,0.670253,0.843666,0.885849,0.805317,0.837324,29.666980,1
2,alpha_1_idx_1,0.632944,0.839695,0.830537,0.849057,0.815457,29.063724,1
7,alpha_1_idx_2,0.641333,0.847331,0.818545,0.878216,0.816538,26.742814,1
0,alpha_10_idx_2,0.629546,0.836892,0.833333,0.840480,0.814409,42.362050,10


In [23]:
# load results 
# results_df = pd.read_csv("/kaggle/input/results-df/results_df.csv").sort_values(by=["vs"], ascending=False)

styled_df = results_df.style \
    .highlight_max(color='lightgreen', axis=0, subset=results_df.columns.difference(['subsample', 'alpha'])) \
    .highlight_min(color='lightcoral', axis=0, subset=results_df.columns.difference(['subsample', 'alpha']))

styled_df

,subsample,mcc,f1,precision,recall,roc_auc,vs,alpha
15,alpha_0_01_idx_1,0.664689,0.855225,0.837305,0.873928,0.830053,12.809858,0.01
14,alpha_0_01_idx_2,0.669845,0.851691,0.861404,0.842196,0.835785,8.428849,0.01
6,alpha_0_01_idx_3,0.623381,0.843750,0.791729,0.903087,0.801976,8.810502,0.01
20,alpha_0_1_idx_2,0.657920,0.851852,0.836364,0.867925,0.827051,21.201905,0.1
16,alpha_0_1_idx_3,0.662145,0.852292,0.843697,0.861063,0.830100,18.217655,0.1
13,alpha_0_1_idx_1,0.654072,0.849916,0.835821,0.864494,0.825336,8.751802,0.1
18,alpha_1_idx_3,0.670253,0.843666,0.885849,0.805317,0.837324,29.666980,1
2,alpha_1_idx_1,0.632944,0.839695,0.830537,0.849057,0.815457,29.063724,1
7,alpha_1_idx_2,0.641333,0.847331,0.818545,0.878216,0.816538,26.742814,1
0,alpha_10_idx_2,0.629546,0.836892,0.833333,0.840480,0.814409,42.362050,10
